In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload() #upload kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"abhivyaktiyadav","key":"3abff0459141716e6adb8a26e694a8b5"}'}

In [3]:
!mkdir -p ~/.kaggle

In [4]:
!cp kaggle.json ~/.kaggle/


In [5]:
!chmod 600 /root/.kaggle/kaggle.json



In [6]:
!kaggle datasets download -d tentotheminus9/seti-data

100% 5.25G/5.26G [00:49<00:00, 83.8MB/s]
100% 5.26G/5.26G [00:49<00:00, 113MB/s] 


In [7]:
!unzip seti-data.zip

Streaming output truncated to the last 5000 lines.
  inflating: primary_small_v3/primary_small_v3/4a1f619c-c3d4-4410-8960-f95a08acb9fc.dat  
  inflating: primary_small_v3/primary_small_v3/4a228d48-fa7e-4acf-b4eb-5e6acdc86b5e.dat  
  inflating: primary_small_v3/primary_small_v3/4a29fd87-442e-45ad-bb59-dce6279a1f80.dat  
  inflating: primary_small_v3/primary_small_v3/4a2c4a5a-2ed0-439f-a4f3-c55de4c6b4c0.dat  
  inflating: primary_small_v3/primary_small_v3/4a46fc04-dd9c-4a9f-9b73-8de844d50c0b.dat  
  inflating: primary_small_v3/primary_small_v3/4a536bee-a0a4-4928-80ab-abf9ab0b4b74.dat  
  inflating: primary_small_v3/primary_small_v3/4a56c8d9-9347-4cc2-83bd-74cfb556c351.dat  
  inflating: primary_small_v3/primary_small_v3/4a63cd37-882d-4cf0-ac6b-e4b703acf780.dat  
  inflating: primary_small_v3/primary_small_v3/4a6d3b8f-2181-479c-8de2-75dce14c91c1.dat  
  inflating: primary_small_v3/primary_small_v3/4a77d659-a957-44c7-a8bd-781f676802c1.dat  
  inflating: primary_small_v3/primary_small_v3/4a

In [8]:
import os
import pandas as pd
import numpy as np

In [9]:
import tensorflow
print(tensorflow.__version__)

2.9.2


In [10]:

import cv2,os
data_path="../content/primary_small/train"
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary
print(label_dict)
print(categories)
print(labels)

{'squiggle': 0, 'squigglesquarepulsednarrowband': 1, 'squarepulsednarrowband': 2, 'narrowbanddrd': 3, 'narrowband': 4, 'brightpixel': 5, 'noise': 6}
['squiggle', 'squigglesquarepulsednarrowband', 'squarepulsednarrowband', 'narrowbanddrd', 'narrowband', 'brightpixel', 'noise']
[0, 1, 2, 3, 4, 5, 6]


## training set

In [11]:
img_size=244
train_data=[]
target=[]

for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:  
            resized=cv2.resize(img,(img_size,img_size))
            #resizing the image  into 244x244, since we need a fixed common size for all the images in the dataset
            train_data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)
        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image



import numpy as np
train_data=np.array(train_data)/255.0
train_data=np.reshape(train_data,(train_data.shape[0],img_size,img_size,3))
target=np.array(target)
from keras.utils import np_utils
train_target=np_utils.to_categorical(target)


print(labels)
     

[0, 1, 2, 3, 4, 5, 6]


In [12]:
train_data.shape

(5600, 244, 244, 3)

In [13]:
train_target.shape

(5600, 7)

test set

In [14]:
img_size=244
data_test=[]
target=[]
data_path1="../content/primary_small/test"

for category in categories:
    folder_path=os.path.join(data_path1,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:  
            resized=cv2.resize(img,(img_size,img_size))
            #resizing the image  into 244x244, since we need a fixed common size for all the images in the dataset
            data_test.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)
        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image



import numpy as np
data_test=np.array(data_test)/255.0
data_test=np.reshape(data_test,(data_test.shape[0],img_size,img_size,3))
target=np.array(target)
from keras.utils import np_utils
target_test=np_utils.to_categorical(target)


print(labels)

[0, 1, 2, 3, 4, 5, 6]


In [15]:
print(data_test.shape)
print(target_test.shape)

(700, 244, 244, 3)
(700, 7)


cnn

In [ ]:
train_data.shape #1200 dataset, 244x244, 3 menandakan RGB

In [ ]:
train_data.shape[1:]

(244, 244, 3)

## splitting data

In [16]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=train_data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(7,activation='softmax'))
#The Final layer with seven outputs for seven categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 242, 242, 200)     5600      
                                                                 
 activation (Activation)     (None, 242, 242, 200)     0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 121, 121, 200)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 119, 119, 100)     180100    
                                                                 
 activation_1 (Activation)   (None, 119, 119, 100)     0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 59, 59, 100)      0         
 2D)                                                    

In [ ]:
print(train_data.shape)
print(train_target.shape)
print(data_test.shape)
print(target_test.shape)
print(data_valid.shape)
print(target_valid.shape)

In [ ]:
#checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
# H = model.fit(train_data,train_target,epochs=50,validation_split=0.2)
#model.compile(loss="categorical_crossentropy", optimizer=opt,
#	metrics=["accuracy"])
H = model.fit(train_data, train_target, validation_data=(data_test, target_test), epochs=15, batch_size=32)

In [ ]:
model.save('model.h5')

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
print(H.history.keys())
# summarize history for accuracy
plt.plot(H.history['accuracy'])
plt.plot(H.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(H.history['loss'])
plt.plot(H.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
N = 15
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="center right")
plt.savefig("CNN_Model")
     